# 导入数据

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [36]:
data_by_artist = pd.read_csv('../2021_ICM_Problem_D_Data/data_by_artist.csv')
data_by_year = pd.read_csv('../2021_ICM_Problem_D_Data/data_by_year.csv')
full_music_data = pd.read_csv('../2021_ICM_Problem_D_Data/full_music_data.csv')
influence_data = pd.read_csv('../2021_ICM_Problem_D_Data/influence_data.csv',encoding='gbk')

In [55]:
data_by_artist.head()

,artist_name,artist_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,duration_ms,popularity,count
0,Frank Sinatra,792507,0.384478,0.238017,0.364288,110.181698,-14.271141,1,5,0.735648,0.020855,0.232106,0.049614,189179.9255,26.004383,1369
1,Vladimir Horowitz,119107,0.343210,0.118844,0.225951,94.900679,-23.193418,1,1,0.990070,0.879508,0.183812,0.043360,266541.1251,3.592378,1207
2,Johnny Cash,816890,0.619803,0.449381,0.680662,115.037747,-11.593104,1,10,0.685637,0.022647,0.242243,0.098216,162279.2672,26.614130,1104
3,Billie Holiday,79016,0.572637,0.201368,0.498934,109.912172,-13.225966,1,5,0.908499,0.013064,0.217727,0.062432,185131.4530,15.621005,1095
4,Bob Dylan,66915,0.512598,0.477932,0.551934,126.160149,-11.184330,1,7,0.562567,0.034211,0.308978,0.064535,256713.4203,30.860806,1092


# 合并表格

In [37]:
influence1 = influence_data[['influencer_id', 'influencer_main_genre']]
influence1.head()

,influencer_id,influencer_main_genre
0,759491,Pop/Rock
1,25462,Electronic
2,66915,Pop/Rock
3,71209,Pop/Rock
4,91438,Pop/Rock


In [38]:
#去掉重复值，即得到每位艺术家所属领域
influence1 = influence1.drop_duplicates()
influence1.head()

,influencer_id,influencer_main_genre
0,759491,Pop/Rock
1,25462,Electronic
2,66915,Pop/Rock
3,71209,Pop/Rock
4,91438,Pop/Rock


In [39]:
influence1.index = influence1['influencer_id']
influence1.head()

,influencer_id,influencer_main_genre
influencer_id,,
759491,759491,Pop/Rock
25462,25462,Electronic
66915,66915,Pop/Rock
71209,71209,Pop/Rock
91438,91438,Pop/Rock


In [40]:
#做一个字典，键为艺术家id，值为艺术家所属领域。
genre_id1 = dict(influence1['influencer_main_genre'])

In [41]:
influence = influence_data[['follower_id', 'follower_main_genre']]
influence.head()

,follower_id,follower_main_genre
0,74,Pop/Rock
1,335,Pop/Rock
2,335,Pop/Rock
3,335,Pop/Rock
4,335,Pop/Rock


In [42]:
#去掉重复值，即得到每位艺术家所属领域
influence = influence.drop_duplicates()
influence.head()

,follower_id,follower_main_genre
0,74,Pop/Rock
1,335,Pop/Rock
21,441,Pop/Rock
39,589,Pop/Rock
42,1098,Pop/Rock


In [43]:
influence.index = influence['follower_id']
influence.head()

,follower_id,follower_main_genre
follower_id,,
74,74,Pop/Rock
335,335,Pop/Rock
441,441,Pop/Rock
589,589,Pop/Rock
1098,1098,Pop/Rock


In [44]:
#做一个字典，键为艺术家id，值为艺术家所属领域。
genre_id = dict(influence['follower_main_genre'])

In [45]:
c={}
for k,v in genre_id.items():
    c[k]=v

for k,v in genre_id1.items():
    c[k]=v  

In [46]:
#得到所有的领域
genres = influence['follower_main_genre'].unique()
genres

array(['Pop/Rock', 'Country', 'Classical', 'Electronic', 'Comedy/Spoken',
       'Easy Listening', 'R&B;', 'Stage & Screen', 'Reggae', 'Blues',
       'New Age', 'Latin', 'Vocal', 'Jazz', 'International', 'Folk',
       'Religious', 'Unknown', 'Avant-Garde', "Children's"], dtype=object)

In [47]:
#将artists_id化为int型
full_music_data['artists_id'] = full_music_data['artists_id'].map(lambda x:x.replace('[', '').replace(']', '').split(',')[0])
full_music_data['artists_id']=full_music_data['artists_id'].map(int)

In [48]:
#在full_music_data中添加一列，为创作该歌曲的艺术家所属领域
full_music_data['genre'] = full_music_data['artists_id'].map(c)

# 艺术家指标

# 处理指标

In [49]:
year=full_music_data[['year']]

In [50]:
data=full_music_data[['energy','valence','mode','danceability',
                      'tempo','loudness','acousticness','liveness','duration_ms','popularity','instrumentalness','genre']]
data.head()

,energy,valence,mode,danceability,tempo,loudness,acousticness,liveness,duration_ms,popularity,instrumentalness,genre
0,0.365,0.131,0,0.600,130.046,-13.083,0.0672,0.0921,437200,54,0.585000,Electronic
1,0.326,0.179,0,0.874,119.620,-13.302,0.0136,0.0993,581008,53,0.148000,Electronic
2,0.531,0.336,0,0.670,139.385,-8.267,0.0156,0.3060,431293,55,0.345000,Electronic
3,0.702,0.850,1,0.761,104.773,-8.523,0.1080,0.0935,304427,34,0.000031,R&B;
4,0.828,0.902,0,0.661,115.078,-12.673,0.2760,0.2870,264933,37,0.000001,R&B;


In [51]:
#处理极小化指标
data['liveness1']=1-data['liveness']

In [52]:
#定义纯度指标
data['purity']=data['liveness1']+data['acousticness']

In [53]:
#定义情感倾向指标
data['emotion']=data['energy']+data['valence']+data['mode']

In [54]:
import pandas as pd
import numpy as np
import math
from numpy import array
#定义熵值法函数
def cal_weight(x):
    '''熵值法计算变量的权重'''
    # 标准化
    x = x.apply(lambda x: ((x - np.min(x)) / (np.max(x) - np.min(x))))
 
    # 求k
    rows = x.index.size  # 行
    cols = x.columns.size  # 列
    k = 1.0 / math.log(rows)
 
    lnf = [[None] * cols for i in range(rows)]
 
    # 矩阵计算--
    # 信息熵
    # p=array(p)
    x = array(x)
    lnf = [[None] * cols for i in range(rows)]
    lnf = array(lnf)
    for i in range(0, rows):
        for j in range(0, cols):
            if x[i][j] == 0:
                lnfij = 0.0
            else:
                p = x[i][j] / x.sum(axis=0)[j]
                lnfij = math.log(p) * p * (-k)
            lnf[i][j] = lnfij
    lnf = pd.DataFrame(lnf)
    E = lnf
 
    # 计算冗余度
    d = 1 - E.sum(axis=0)
    # 计算各指标的权重
    w = [[None] * 1 for i in range(cols)]
    for j in range(0, cols):
        wj = d[j] / sum(d)
        w[j] = wj
        # 计算各样本的综合得分,用最原始的数据
    return w

In [22]:
w = cal_weight(data[['danceability','tempo','loudness']])
w

[0.5854533265576414, 0.35210331038254183, 0.06244336305981675]

In [23]:
data['rhythm']=w[0]*data['danceability']+w[1]*data['tempo']+w[2]*data['loudness']

In [24]:
data2=data[['emotion','rhythm','purity','duration_ms','popularity','instrumentalness','genre']]
data3['year']=year['year']
data3.to_csv('六个指标标准化前.csv')

NameError: name 'data3' is not defined

In [ ]:
data4=data[['emotion','rhythm','purity','duration_ms','popularity','instrumentalness']]
data5=data4.apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))
data5['genre']=data['genre']
data5['year']=year['year']
data5.to_csv('六个指标标准化后.csv')